In [1]:
# !pip install openai

In [2]:
openai_key = open('../open_ai.key','r').read()
# print(openai_key)

In [3]:
import openai
openai.api_key = openai_key
def completion(new_message_text:str, settings_text:str = '', past_messages:list = []):
    """
    new_message_text: str ユーザ入力
    settings_text: str システム設定（ChatGPTのふるまいを指定する文言）
    past_messages: list ユーザとシステムのやりとりの履歴
    """
    if len(past_messages) == 0 and len(settings_text) != 0:
        system = {"role": "system", "content": settings_text}
        past_messages.append(system)
    new_message = {"role": "user", "content": new_message_text}
    past_messages.append(new_message)

    result = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=past_messages,
        max_tokens=2048,
    )
    response_message = {"role": "assistant", "content": result.choices[0].message.content}
    past_messages.append(response_message)
    response_message_text = result.choices[0].message.content
    return response_message_text, past_messages

In [4]:
system_settings = """レシピテキストをJson形式のレシピデータを生成してください。
では、会話を開始します。
"""

In [5]:
new_message, messages = completion("こんにちは", system_settings, [])
print(new_message)


こんにちは。レシピデータを生成するためには、どのような情報が必要ですか？


In [6]:
import json
recipe_format = json.dumps({
    "title": "",
    "author": "",
    "ingredients": [
        {"name": "", "amount": "", "gram": 0, "ml": 0},
    ],
    "appliances": [""],
    "steps": [""],
    "cooking_time": "",
    "junre": [""],
    "nutrition": {
        "calorie": 0.0,
        "protein": 0.0,
        "fat": 0.0,
        "carbohydrate": 0.0,
        "sodium": 0.0,
    },
})

In [7]:
from bs4 import BeautifulSoup
import requests

url = "https://www.lettuceclub.net/recipe/dish/22978/"
response = requests.get(url)
soup = BeautifulSoup(response.content, "html.parser")
recipe_text = soup.get_text().replace('\n\n','')
# print(recipe_text)

In [8]:
# 文字数制限があるため、不要部分をカット
end = 'レシピランキング'
recipe_text = recipe_text[:recipe_text.find(end)]

new_message, messages = completion(
    f"以下のレシピを{recipe_format}に従って整形してください。\n\n{recipe_text}",
    system_settings, messages
)

In [9]:
print(new_message)

こちらのレシピをJson形式に整形したデータは以下のようになります。

```json
{
  "title": "簡単タコライス",
  "author": "Mako(多賀正子)",
  "ingredients": [
    {"name": "合いびき肉", "amount": "180g", "gram": 180},
    {"name": "トッピング用チーズ", "amount": "適量"},
    {"name": "玉ねぎのみじん切り", "amount": "1/3個分"},
    {"name": "トマト", "amount": "1/2個"},
    {"name": "にんにくの粗みじん切り", "amount": "１片分"},
    {"name": "レタス", "amount": "1/3個"},
    {"name": "トルティーヤチップス", "amount": "適量"},
    {"name": "温かいご飯", "amount": "どんぶり２杯分"},
    {"name": "トマトケチャップ"},
    {"name": "オリーブ油", "amount": "大さじ１"},
    {"name": "カレー粉", "amount": "大さじ2/3〜１"},
    {"name": "ウスターソース", "amount": "大さじ１"},
    {"name": "しょうゆ", "amount": "大さじ1/2"},
    {"name": "こしょう"}
  ],
  "appliances": ["フライパン"],
  "steps": [
    "トマトは１cm角に切って、ケチャップ適量であえる。",
    "フライパンにオリーブ油大さじ１を熱し、玉ねぎとにんにくを炒める。香りが立ったらひき肉を加えてほぐすようにして炒め、カレー粉大さじ2/3〜１、ケチャップ大さじ２、ソース大さじ１、しょうゆ大さじ1/2、こしょう少々を加えて全体に味がなじむまで炒める。",
    "器にご飯を盛り、レタスを食べやすい大きさにちぎってのせ、チーズを散らす。２とトマトものせ、トルティーヤチップスを砕いてのせる。好みで粗びき黒こしょうやタバスコをふっても。"
  ],
  "cookin

In [10]:
import re
pattern = r'\{.*\}'

# 正規表現を使用してJSON部分のみを抽出する
match = re.search(pattern, new_message, re.DOTALL)
if match:
    json_string = match.group()
    json_obj = json.dumps(json.loads(json_string),indent=4,ensure_ascii=False)
    print(json_obj)

{
    "title": "簡単タコライス",
    "author": "Mako(多賀正子)",
    "ingredients": [
        {
            "name": "合いびき肉",
            "amount": "180g",
            "gram": 180
        },
        {
            "name": "トッピング用チーズ",
            "amount": "適量"
        },
        {
            "name": "玉ねぎのみじん切り",
            "amount": "1/3個分"
        },
        {
            "name": "トマト",
            "amount": "1/2個"
        },
        {
            "name": "にんにくの粗みじん切り",
            "amount": "１片分"
        },
        {
            "name": "レタス",
            "amount": "1/3個"
        },
        {
            "name": "トルティーヤチップス",
            "amount": "適量"
        },
        {
            "name": "温かいご飯",
            "amount": "どんぶり２杯分"
        },
        {
            "name": "トマトケチャップ"
        },
        {
            "name": "オリーブ油",
            "amount": "大さじ１"
        },
        {
            "name": "カレー粉",
            "amount": "大さじ2/3〜１"
        },
        {
            "name": "ウスターソース",
